# Journalist Classification with TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\s

import the dataset

In [2]:
articles = pd.read_csv("articles.csv")

Split into X and y

In [3]:
y = articles["author"]
X = articles["article"]

Define function next_batch (obtained stack_overflow)

In [4]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


Now, we've had all ingredients to build a logistic or softmax classification. Let's build a simple softmax classification.

In [5]:
# Importing necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# 80-20 splitting the dataset (80%->Training and 20%->Validation)

X_train, X_test, y_train, y_test = train_test_split(X, y
                                   ,test_size=0.2, random_state=1234)
y_train = np.array(y_train)
y_test = np.array(y_test)

# defining the bag-of-words transformer on the text-processed corpus # i.e., text_process() declared in II is executed...
bow_transformer=CountVectorizer(max_features = 2500).fit(X_train)
# transforming into Bag-of-Words and hence textual data to numeric..
text_bow_train=bow_transformer.transform(X_train).toarray()#ONLY TRAINING DATA

# transforming into Bag-of-Words and hence textual data to numeric..
text_bow_test=bow_transformer.transform(X_test).toarray()#TEST DATA

In [8]:
tf.reset_default_graph()

n_inputs = 1 * 2500
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 58
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name='X')
y = tf.placeholder(tf.int64, shape=[None], name='y')

W = tf.Variable(tf.truncated_normal([n_inputs, n_outputs], stddev=0.02), name='weights')
b = tf.Variable(tf.zeros([n_outputs]), name='biases')

logits = tf.add(tf.matmul(X, W), b, name='logits')

with tf.name_scope('evaluation'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='xentropy')
    loss = tf.reduce_mean(xentropy, name='loss')
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

with tf.name_scope("train"):
    grad_W, grad_b = tf.gradients(loss, [W, b])
    update_W = tf.assign(W, W - learning_rate * grad_W)
    update_b = tf.assign(b, b - learning_rate * grad_b)

init = tf.global_variables_initializer()

In [ ]:
n_epochs = 200
batch_size = 100

with tf.Session() as sess:
    init.run()
    print("Epoch\tTrain accuracy\tTest accuracy")
    for epoch in range(n_epochs):
        for iteration in range(len(text_bow_train) // batch_size):
            X_batch, y_batch = next_batch(batch_size,text_bow_train,y_train)
            sess.run([update_W, update_b], feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: text_bow_test, y: y_test})
        print('%d\t%f\t%f' % (epoch, acc_train, acc_test))

The accuracy was not bad, roughly 64%. Compared to the 49% from the Naive Bayes, this is a tremendeous increase. However, Now let's try it with tensorflow

In [10]:
import tensorflow as tf
tf.reset_default_graph()

n_inputs = 1 * 2500
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 58
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name='X')
y = tf.placeholder(tf.int64, shape=[None], name='y')

W = tf.Variable(tf.truncated_normal([n_inputs, n_outputs], stddev=0.02), name='weights')
b = tf.Variable(tf.zeros([n_outputs]), name='biases')

logits = tf.add(tf.matmul(X, W), b, name='logits')

with tf.name_scope('evaluation'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='xentropy')
    loss = tf.reduce_mean(xentropy, name='loss')
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [11]:
n_epochs = 200
batch_size = 100

with tf.Session() as sess:
    init.run()
    print("Epoch\tTrain accuracy\tTest accuracy")
    for epoch in range(n_epochs):
        for iteration in range(len(text_bow_train) // batch_size):
            X_batch, y_batch = next_batch(batch_size,text_bow_train,y_train)
            sess.run([update_W, update_b], feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: text_bow_test, y: y_test})
        print('%d\t%f\t%f' % (epoch, acc_train, acc_test))

Epoch	Train accuracy	Test accuracy


ValueError: Fetch argument <tf.Tensor 'train/Assign:0' shape=(2500, 58) dtype=float32_ref> cannot be interpreted as a Tensor. (Tensor Tensor("train/Assign:0", shape=(2500, 58), dtype=float32_ref) is not an element of this graph.)